In [ ]:
import json

from allauth.socialaccount.app_settings import QUERY_EMAIL
from allauth.socialaccount.providers.oauth.client import OAuth
from allauth.socialaccount.providers.oauth.views import (
    OAuthAdapter,
    OAuthCallbackView,
    OAuthLoginView,
)

from .provider import TwitterProvider


In [ ]:
class TwitterAPI(OAuth):
    """
    Verifying twitter credentials
    """

    _base_url = "https://api.twitter.com/1.1/account/verify_credentials.json"
    url = _base_url + "?include_email=true" if QUERY_EMAIL else _base_url

    def get_user_info(self):
        user = json.loads(self.query(self.url))
        return user


In [ ]:
class TwitterOAuthAdapter(OAuthAdapter):
    provider_id = TwitterProvider.id
    request_token_url = "https://api.twitter.com/oauth/request_token"
    access_token_url = "https://api.twitter.com/oauth/access_token"
    # Issue #42 -- this one authenticates over and over again...
    # authorize_url = 'https://api.twitter.com/oauth/authorize'
    authorize_url = "https://api.twitter.com/oauth/authenticate"

    def complete_login(self, request, app, token, response):
        client = TwitterAPI(request, app.client_id, app.secret, self.request_token_url)
        extra_data = client.get_user_info()
        return self.get_provider().sociallogin_from_response(request, extra_data)


In [ ]:
oauth_login = OAuthLoginView.adapter_view(TwitterOAuthAdapter)
oauth_callback = OAuthCallbackView.adapter_view(TwitterOAuthAdapter)

